In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import polars as pl
from talk_reader import TalkReader
from glob import glob
import os
import shutil
from pathlib import Path

# metadata_df = pl.read_csv("data/metadata/metadata.csv")
# len(metadata_df)
# for path in glob("data/*.docx"):
#     talk_reader = TalkReader(
#         path, "/Users/llohann/Python/amruta/data/metadata/metadata.csv"
#     )
#     talk_reader.dump_metadata()
os.environ["DEBUG"] = "0"
original_files_list = list(glob("data/original_files/*.docx"))
# first_file = original_files_list[0]

_save_folder = "data/test/output"
metadata_path = "data/test/metadata.csv"
os.remove(metadata_path)
shutil.rmtree(_save_folder, ignore_errors=True)

parsing_that_went_wrong = []

for file in original_files_list:
    save_folder = Path(_save_folder) / Path(file).name[:4]
    talk_reader = TalkReader(file, metadata_path, save_folder=save_folder)
    metadata = talk_reader.process_document()
    if talk_reader.parsing_that_went_wrong:
        parsing_that_went_wrong.extend(talk_reader.parsing_that_went_wrong)

In [ ]:
output_files = list(Path(_save_folder).rglob("*.docx"))
num_files = len(output_files)
print(f"Number of files in '{save_folder}': {num_files}")


In [ ]:
from tqdm import tqdm
import re

data = []


def extract_lines_starting_with_number(text: str) -> list[str]:
    return re.findall(r"^\d\d?[^\.:]*\d\d\s*$", text, re.MULTILINE)


for file in tqdm(output_files):
    talk_reader = TalkReader(file, metadata_path, save_folder=save_folder)
    text = talk_reader._get_all_text()
    lines_starting_with_number = extract_lines_starting_with_number(text)
    data.append(
        {"file": file, "lines_starting_with_number": lines_starting_with_number}
    )
data = pl.DataFrame(data)


In [ ]:
data_filtered = (
    data.with_columns(pl.col("lines_starting_with_number").list.len().alias("n_lines"))
    .filter(pl.col("n_lines") > 1)
    .sort("n_lines")
)

with pl.Config() as cfg:
    cfg.set_tbl_rows(100)
    cfg.set_fmt_str_lengths(100)
    display(data_filtered)

# Reading the metadata

In [ ]:
metadata_df = pl.read_csv(metadata_path)
metadata_df.write_excel(metadata_path.replace(".csv", ".xlsx"))
len(metadata_df)

In [ ]:
metadata_df.filter(pl.col("Comments").is_not_null())